# SENTIMENT BASED PLAYSTATION

In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from nltk.corpus import stopwords
import speech_recognition as sr
from deepface import DeepFace
from datetime import datetime
from time import sleep
from gtts import gTTS
import numpy as np
import subprocess
import statistics
import playsound
import keyword
import pyaudio
import pyglet
import nltk
import cv2
import time
import os 
import re
import warnings
import ctypes
import time
from imutils.video import VideoStream
import argparse
import imutils
import pyttsx3
#nltk.download('stopwords')
warnings.filterwarnings("ignore") 
#nltk.download('punkt')
emotion_list=[]

In [2]:
def control():
    
    
    #direct keys
    SendInput = ctypes.windll.user32.SendInput


    W = 0x11
    A = 0x1E
    S = 0x1F
    D = 0x20

    # C struct redefinitions 
    PUL = ctypes.POINTER(ctypes.c_ulong)
    class KeyBdInput(ctypes.Structure):
        _fields_ = [("wVk", ctypes.c_ushort),
                    ("wScan", ctypes.c_ushort),
                    ("dwFlags", ctypes.c_ulong),
                    ("time", ctypes.c_ulong),
                    ("dwExtraInfo", PUL)]

    class HardwareInput(ctypes.Structure):
        _fields_ = [("uMsg", ctypes.c_ulong),
                    ("wParamL", ctypes.c_short),
                    ("wParamH", ctypes.c_ushort)]

    class MouseInput(ctypes.Structure):
        _fields_ = [("dx", ctypes.c_long),
                    ("dy", ctypes.c_long),
                    ("mouseData", ctypes.c_ulong),
                    ("dwFlags", ctypes.c_ulong),
                    ("time",ctypes.c_ulong),
                    ("dwExtraInfo", PUL)]

    class Input_I(ctypes.Union):
        _fields_ = [("ki", KeyBdInput),
                     ("mi", MouseInput),
                     ("hi", HardwareInput)]

    class Input(ctypes.Structure):
        _fields_ = [("type", ctypes.c_ulong),
                    ("ii", Input_I)]

    # Actuals Functions

    def PressKey(hexKeyCode):
        extra = ctypes.c_ulong(0)
        ii_ = Input_I()
        ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
        x = Input( ctypes.c_ulong(1), ii_ )
        ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

    def ReleaseKey(hexKeyCode):
        extra = ctypes.c_ulong(0)
        ii_ = Input_I()
        ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
        x = Input( ctypes.c_ulong(1), ii_ )
        ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

    if __name__ == '__main__':
        PressKey(0x11)
        time.sleep(1)
        ReleaseKey(0x11)
        time.sleep(1)
    

    # define the lower and upper boundaries of the "orange" object in the HSV color space
    orangeLower = np.array([0, 129, 100])
    orangeUpper = np.array([180,255,255])

    vs = VideoStream(src=0).start()

    # allow the camera or video file to warm up
    time.sleep(2.0)
    initial = True
    flag = False
    current_key_pressed = set()
    circle_radius = 30
    windowSize = 160
    lr_counter = 0
    # keep looping
    while True:
        keyPressed = False
        keyPressed_lr = False
        # grab the current frame
        frame = vs.read()
        height,width = frame.shape[:2]

        # resize the frame, blur it, and convert it to the HSV color space
        frame = imutils.resize(frame, width=600)
        blurred = cv2.GaussianBlur(frame, (11, 11), 0)
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

        # crteate a mask for the orange color and perform dilation and erosion to remove any small
        # blobs left in the mask
        mask = cv2.inRange(hsv, orangeLower, orangeUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # find contours in the mask and initialize the current
        # (x, y) center of the orange object

        # divide the frame into two halves so that we can have one half control the acceleration/brake 
        # and other half control the left/right steering.
        left_mask = mask[:,0:width//2,]
        right_mask = mask[:,width//2:,]

        #find the contours in the left and right frame to find the center of the object
        cnts_left = cv2.findContours(left_mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts_left = imutils.grab_contours(cnts_left)
        center_left = None

        cnts_right = cv2.findContours(right_mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts_right = imutils.grab_contours(cnts_right)
        center_right = None

        # only proceed if at least one contour was found
        if len(cnts_left) > 0:
            # find the largest contour in the mask, then use
            # it to compute the minimum enclosing circle and centroid
            c = max(cnts_left, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            # find the center from the moments 0.000001 is added to the denominator so that divide by 
            # zero exception doesn't occur
            center_left = (int(M["m10"] / (M["m00"]+0.000001)), int(M["m01"] / (M["m00"]+0.000001)))

            # only proceed if the radius meets a minimum size
            if radius > circle_radius:
                # draw the circle and centroid on the frame,
                cv2.circle(frame, (int(x), int(y)), int(radius),
                    (0, 255, 255), 2)
                cv2.circle(frame, center_left, 5, (0, 0, 255), -1)

                #the window size is kept 160 pixels in the center of the frame(80 pixels above the center and 80 below)
                if center_left[1] < (height/2 - windowSize//2):
                    cv2.putText(frame,'LEFT',(20,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
                    PressKey(A)
                    current_key_pressed.add(A)
                    keyPressed = True
                    keyPressed_lr = True
                elif center_left[1] > (height/2 + windowSize//2):
                    cv2.putText(frame,'RIGHT',(20,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
                    PressKey(D)
                    current_key_pressed.add(D)
                    keyPressed = True
                    keyPressed_lr = True

        # only proceed if at least one contour was found
        if len(cnts_right) > 0:
            c2 = max(cnts_right, key=cv2.contourArea)
            ((x2, y2), radius2) = cv2.minEnclosingCircle(c2)
            M2 = cv2.moments(c2)
            center_right = (int(M2["m10"] / (M2["m00"]+0.000001)), int(M2["m01"] / (M2["m00"]+0.000001)))
            center_right = (center_right[0]+width//2,center_right[1])

            # only proceed if the radius meets a minimum size
            if radius2 > circle_radius:
                # draw the circle and centroid on the frame,
                cv2.circle(frame, (int(x2)+width//2, int(y2)), int(radius2),
                    (0, 255, 255), 2)
                cv2.circle(frame, center_right, 5, (0, 0, 255), -1)
                if center_right[1] < (height//2 - windowSize//2):
                    cv2.putText(frame,'UP',(200,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
                    PressKey(W)
                    keyPressed = True
                    current_key_pressed.add(W)
                elif center_right[1] > (height//2 + windowSize//2):
                    cv2.putText(frame,'DOWN',(200,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255))
                    PressKey(S)
                    keyPressed = True
                    current_key_pressed.add(S)


        # show the frame to our screen
        frame_copy = frame.copy()
        frame_copy = cv2.rectangle(frame_copy,(0,height//2 - windowSize//2),(width,height//2 + windowSize//2),(255,0,0),2)
        cv2.imshow("Frame", frame_copy)

        #We need to release the pressed key if none of the key is pressed else the program will keep on sending
        # the presskey command 
        if not keyPressed and len(current_key_pressed) != 0:
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()

        #to release keys for left/right with keys of up/down remain pressed   
        if not keyPressed_lr and ((A in current_key_pressed) or (D in current_key_pressed)):
            if A in current_key_pressed:
                ReleaseKey(A)
                current_key_pressed.remove(A)
            elif D in current_key_pressed:
                ReleaseKey(D)
                current_key_pressed.remove(D)

        key = cv2.waitKey(1) & 0xFF

        # if the 'q' key is pressed, stop the loop
        if key == ord("q"):
            break


    vs.stop() 
    # close all windows
    cv2.destroyAllWindows()

## EMOTION RECOGNITION 

In [3]:
timeout = 45   # [seconds]
def emotion():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    timeout_start = time.time()
    cap=cv2.VideoCapture(0)

    while True:
            ret,frame=cap.read()
            result=DeepFace.analyze(frame,actions= ['emotion'] )
            gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.1,4)


            for(x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            font=cv2.FONT_HERSHEY_SIMPLEX

            emotion_list.append(result['dominant_emotion'])
            cv2.putText(frame,result['dominant_emotion'],(50,50), font,3,(0,0,255),2,cv2.LINE_4)
            cv2.imshow('Original video',frame)
            print(result['dominant_emotion'])
            k = cv2.waitKey(30) & 0xff
            if k == 27: # press 'ESC' to quit
                break
            elif time.time() >= timeout_start + timeout:
                break
    cap.release()
    cv2.destroyAllWindows()
    mode_emotion=statistics.mode(emotion_list)
    return mode_emotion

## TEXT TO SPEECH FUNCTION

#### gTTS:
gTTs or google text to speech is a python library and CLI tool to interface with google translate text to speech API

#### Features:
 • Customizable speech-specific sentence tokenizer that allows for unlimited lengths of text to be read, all while keeping proper intonation, abbreviations, decimals and more
 
•Customizable text pre-processors which can, for example, provide pronunciation corrections;

In [4]:
def function_speak(spk):
    tts = gTTS(text=spk, lang='en' , slow=False )
    filename = 'voice.mp3'
    tts.save(filename)
    function_assistant_speak(1)
    music=pyglet.media.load(filename)
    music.play()
    #prevent from killing
    time.sleep(music.duration)
    #remove temperory file
    os.remove(filename) 

    


In [5]:
def function_assistant_speak(val):
    if val == 1:
        filename='Start_listening.mp3'
    elif val == 2:
        filename = 'Start_processing.mp3'
    else:
        filename ='Start_speaking.mp3'
    
    music = pyglet.media.load(filename, streaming=False)
    music.play()
    time.sleep(music.duration)

## SPEECH TO TEXT FUNCTION

In [6]:
#speech to text function 
def get_audio(stop):
    while True:
        #creating object
        r=sr.Recognizer()
        #taking access from microphone
        with sr.Microphone() as source:
            print("TALK")
            function_assistant_speak(1)
            audio=r.listen(source)
            function_assistant_speak(2)
            print("Time over, thanks")
            
        try:
            said=r.recognize_google(audio)
            print("Text : "+said)
            #tokenising the text
            x=nltk.word_tokenize(said) 
            stopword = set(stop)
            #removing stopwords
            filtered_sentence = [w for w in x if not w in stopword]
            return filtered_sentence
            break
        except:
            function_assistant_speak(3)
            function_speak("Sorry, I did not get that. Kindly repeat.")
            continue

In [7]:
#created another function for stt with specified stop words
def get_audio2(check):
    if(check==True):
        stop = stopwords.words("english")
    else:
        stop=[]
    return get_audio(stop)   

In [8]:
#function to convert list to string
def listToString(s): 
    # initialize an empty string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
        str1 += ele 
    return str1 

## FUNCTION TO CALL AAPS FROM SYSTEM
#### SUBPROCESS:
The subprocess module present in Python(both 2.x and 3.x) is used to run new applications or programs through Python code by creating new processes. It also helps to obtain the input/output/error pipes as well as the exit codes of various commands.

To execute different programs using Python two functions of the subprocess module are used:
##### 1.subprocess.check_call(args, *, stdin=None, stdout=None, stderr=None, shell=False)
##### 2.subprocess.check_output(args, *, stdin=None, stderr=None, shell=False, universal_newlines=False)

In [9]:
def apps():
    while (True):
        stop=stopwords.words("english") + ["hi","name","open","'s"]
        x=get_audio(stop)
        
        if('calculator' in x):
            function_speak("Opening calculator!")
            subprocess.call("C:\Windows\System32\calc.exe")
            function_speak("Calculator opened")
            break;
        else:
            function_speak("Sorry, I did not get that. Kindly repeat.")
            continue

In [10]:
#returns current time
def get_time():
    now = datetime.now()
    #getting time in specific time fromat
    current_time = now.strftime("%H:%M:%S")
    function_speak(current_time)

## FUNCTION TO CALL GAMES FROM SYSTEM

In [11]:
def games():
    while (True):
        stop=stopwords.words("english") + ["hi","name","open","'s","I","want","play","need"]
        g=get_audio(stop)
        if ('speed' in g) or ('Speed' in g):
            #opening folder
            
            os.startfile(r'C:\\Program Files (x86)\\R.G. Mechanics\\Need for Speed Most Wanted')
            function_speak("Taking you to a! new gaming world")
            control()
            break;
        else:
            function_speak("Sorry, I did not get that. Kindly repeat.")
            continue


## FUNCTION TO FETCH URL

In [12]:
def url(text): 
    fin=re.findall('(?:[-\w.]|(?:%[\da-fA-F]{2}))+',text)
    for i in fin:
        if "." in i:
            return i        
    print(fin)
    if "." in fin:
        return fin[fin.index(".")-1]+"."+fin[fin.index(".")+1]
        #break
    elif "dot" in fin:
        return fin[fin.index("dot")-1]+"."+fin[fin.index("dot")+1]
        #break
    else:
        return text

## PINGING THE WEBSITES USING SELENIUM

In [13]:
def fetch(z):
    #setting up the chrome driver
    driver=webdriver.Chrome(executable_path="chromedriver.exe")
    driver.get("http://"+z.strip()+"/")

def ping():
    l=get_audio2(False)
    print(l)
    s = " "
    # joins elements of list1 by ' ' 
    # and stores in sting s 
    s = s.join(l)
    y = s.lower()
    z = url(y)
    function_speak("Is this website correct "+z+". yes or no?")
    a=""
    l=get_audio2(False)
    a=a.join(l)
    a=a.lower()
    a=a.strip()
    if (a == "yes"):
        function_speak("establishing a connection!")
        fetch(z)
    else:
        function_speak("Speak again")
        ping()

## FUNCTION TO LAUNCH OUR PLAYSTATION

In [14]:
dict1={}
def main_function():
        if(len(dict1.keys())!=0):
            function_assistant_speak(3)
            function_speak("Hi "+s+". What do you want to do ?")
            get_audio2(False)
        else:
            stop=stopwords.words("english") + ["hi","name","open","'s","assistant"]
            function_speak("Good afternoon! Rajiv sir! and Digvijay sir")
            function_speak("NAMASHKAR! AAPKA SWAGAT HAI hamari DILCHASP DUNIYA ME")
            function_speak("Welcome to Playstation! Analyzing your emotion ")
            #returning the sentiment analyzed by emotion function 
            sentiment=emotion()
            function_speak("May I know your good name")
            l = get_audio(stop)
            if(len(l)==0):
                l=get_audio(stop)
            s = " "
            s = s.join(l) 
            dict1['name']=s
            #creating loop to call our features as many times as possible
            while(True):
                function_speak("Hi "+s+". What can i do for you ?")
                a= get_audio(stop)
                if(len(a)==0):
                    a=get_audio(stop)
                    
                if('app' in a)or('apps' in a)or('Apps' in a)or('App' in a)or('aap' in a)or('Aap' in a):
                    function_speak("Which app do you want to open!")
                    apps()
                    
                elif('game'in a)or('Game'in a)or('Games'in a)or('games'in a):
                    function_speak("You seem to be"+sentiment+"You should play need for speed but Which game do you want to play!")
                    games()
                    
                elif('website'in a)or('websites'in a)or('Website'in a)or('Websites'in a):
                    function_speak("please name the website you want to visit!")
                    ping()
                    
                elif('time'in a)or('times'in a)or('Time'in a)or('Times'in a):
                    get_time()
                    
                elif('close'in a)or('Close'in a):
                    function_speak("NOW! closing the assistant!")
                    function_speak("Thank you to Dhruv. AAshish. yash. mohit. and jahnvi. for making me. ")
                    function_speak("Thank you Digvijay Sir for guiding them! Goodbye") 
                    
                    break
                    
                else:
                    function_speak("Sorry, I did not get that. Kindly repeat.")

In [15]:
main_function()

neutral
neutral
neutral
happy
happy
happy
happy
happy
happy
happy
happy
happy
neutral
happy
happy
happy
TALK
Time over, thanks
Text : my name is Yash
TALK
Time over, thanks
Text : I want you to open an app for me
TALK
Time over, thanks
Text : calculator
TALK
Time over, thanks
Text : I want you to open an website
TALK
Time over, thanks
Text : youtube.com
['youtube.com']
TALK
Time over, thanks
Text : yes
TALK
Time over, thanks
Text : ab Mera game khelne ka man ho raha hai to Tum game laga do koi
TALK
Time over, thanks
Text : theek hai Need for Speed khelta hai
TALK
Time over, thanks
Text : ok I want you to close the assistant now
